In [1]:
import re
import spacy
import string
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')
nerModel = spacy.load('./NER_model')


def textfilter(txt):
    chars = re.escape(string.punctuation)
    return (re.sub(r'[' + chars + ']', '', txt.lower()))


def compare(text1, text2):
    cnt = 0
    for i in text1.split(' '):
        for j in text2.split(' '):
            if i == j:
                cnt += 1
    if cnt == len(text1.split(' ')):
        return True
    else:
        return False


def filtersimilar(txt):
    i = 0
    while i < len(txt) - 1:
        text1 = textfilter(txt[i])
        j = i + 1
        flag = 0
        while j < len(txt):
            text2 = textfilter(txt[j])
            if len(text1) <= len(text2):
                if compare(text1, text2):
                    del txt[i]
                    flag = 1
            else:
                if compare(text2, text1):
                    del txt[j]
                    flag = 1

            if flag == 1:
                i = 0
                break
            else:
                j = j + 1
        i=i+1

    return txt

def getNamedEntity(text):
    
    txt = re.sub(r'[0-9]','',text)
    doc = nerModel(txt)
    tempORG = []
    tempPERSON = []
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            tempORG.append(ent.text)
        if(ent.label_=='PERSON'):
            tempPERSON.append(ent.text)
    #print(tempORG)
    tempORG = list(set(tempORG))
    tempPERSON = list(set(tempPERSON))

    tempORG = filtersimilar(tempORG)
    tempPERSON = filtersimilar(tempPERSON)
    
    PERSON = []
    for person in tempPERSON:
        if len(tempPERSON)>0:
            temp = dict()
            index_txt = get_index(person, text)
            temp['index']= index_txt[0]
            temp['value']= index_txt[1]
            PERSON.append(temp)
    
    ORG = []
    for org in tempORG:
        if len(tempORG)>0:
            temp = dict()
            index_txt = get_index(org, text)
            temp['index']= index_txt[0]
            temp['value']= index_txt[1]
            ORG.append(temp)

    returnvalue = {}
    if(len(tempORG)>0):
        returnvalue['ORG']= ORG
    else:
        returnvalue['ORG']=None
        
    if(len(tempPERSON)>0):
        returnvalue['PERSON']= PERSON
    else:
        returnvalue['PERSON']= None
    
    return returnvalue

/data/mmortgage/NERAPI/NER/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_index(txt2find, txt):
      
    for i,character in enumerate(txt):
        flag=0
        start=0
        temp=''
        if character == txt2find[0]:
            start = i
            j=i
            k=0
            while k<len(txt2find):
                if((txt[j]==txt2find[k])&(j<len(txt)-1)):
                    temp=temp+txt[j]
                    j=j+1
                    k=k+1
                    flag=1
                else:
                    flag=0
                    end=j
                    break
        if ((flag==1)&(len(temp)==len(txt2find))):
            return([start, start+len(temp)],txt2find)
    return(None)

In [4]:
import json
with open('./OCR_Files/11072554.txt', 'r') as f:
    data = json.loads(f.read())

In [5]:
txt = ''
for pages in data['analyzeResult']['readResults']:
    for line in pages['lines']:
        txt = txt+' '+line['text']
print(txt)
getNamedEntity(txt)

 Company Code Loc/Dept Number Page KD / EQK 25088257 01/100 1769505 1 of 1 Earnings Statement ADP Masters Smith & Wisby PA 4811 Beach Blvd Ste 300 Period Starting: 02/15/2021 Jacksonville, FL 32207-4867 Period Ending: 02/27/2021 Pay Date: 02/26/2021 Taxable Marital Status: Single Exemptions/Allowances; Tax Override: Federal: 15 Federal: Gary M Huggett 700.00 Flat State: 0 State: 120 29Th Avenue South Local: Local: Jacksonville Beach, FL 32250 Social Security Number: xxx-xx-XXXX Eamings rate hours/units this period year to date Regular 0.00 5250.00 21000.00 Gross Pay $5,250.00 $21,000.00 Deposits Statutory Deductions this period year to date account number transit/ABA amount Federal Income -700.00 2800.00 XXXXXX1875 xxxxxxxxX 3660.22 Social Security -313.87 1255.50 Medicare -73.41 293.63 Voluntary Deductions this period year to date *SIMPLE IRA $ -315.00 1260.00 *HSA CAF125 pre-tax $ -187.50 750.00 Net Pay $3,660.22 Your federal taxable wages this period are $4,747.50 Excluded from Fede

{'ORG': [{'index': [98, 122], 'value': 'Masters Smith & Wisby PA'}],
 'PERSON': [{'index': [339, 353], 'value': 'Gary M Huggett'}]}

In [6]:
txt = """44444 For Official Use Only Safe, accurate, Tse file Visit the IRS website OMB No. 1545-0008 FAST! Use at www.irs.gov. a Employer’s name, 
address, and ZIP code c Tax year/Form corrected d Employee’s correct SSN FACET WEALTH INC FACET WEALTH INC 2020 / W-2 210-56-2343 509 S 
EXETER ST STE 202 e Corrected SSN and/or name (Check this box and complete boxes f and/or BALTIMORE MD 21202 g if incorrect on form 
previously filed.) Complete boxes f and/or g only if incorrect on form previously filed f Employee’s previously reported SSN b 
Employer's Federal EIN g Employee’s previously reported name 81-3453207 h Employee’s first name and initial Last name Suff. 
DANIELLE COLE Note. Only complete money fields that are being corrected (exception: for 5706 S. INDIANAPOLIS AVE. corrections involving MQGE, 
see the General Instructions for Forms W-2 TULSA, OK 74135 and W-3, under Specific Instructions for Form W-2c, boxes 5 and 6). 
i Employee’s address and ZIP code Previously reported Correct information Previously reported Correct information 1 Wages, tips, 
other compensation 1 Wages, tips, other compensation 2 Federal income tax withheld 2 Federal income tax withheld 103016.61 103430.46 3 
Social security wages 3 Social security wages 4 Social security tax withheld 4 Social security tax withheld 122516.61 122930.46 7596.03 
7621.69 5 Medicare wages and tips 5 Medicare wages and tips 6 Medicare tax withheld 6 Medicare tax withheld 122516.61 122930.46 1776.49 
1782.49 7 Social security tips 7 Social security tips 8 Allocated tips 8 Allocated tips 9 9 10 Dependent care benefits 10 Dependent care 
benefits 11 Nonqualified plans 11 Nonqualified plans 12a See instructions for box 12 12a See instructions for box 12 C C d o o d 13 
Statutory Retirement Third-party 13 Statutory Retirement Third-party 12b e 12b e employee plan sick pay employee plan sick 
pay C C o o d d e e 14 Other (see instructions) 14 Other (see instructions) 12c C 12c C o o d d e e 12d 12d o C o C e d d e 
State Correction Information Previously reported Correct information Previously reported Correct information 15 State 15 State 15 State 15 
State OK OK Employer’s state ID number Employer’s state ID number Employer’s state ID number Employer’s state ID number 
WTH-15489585-02 WTH-15489585-02 16 State wages, tips, etc. 16 State wages, tips, etc. 16 State wages, tips, etc. 16 State wages, tips, etc. 
34929.67 35343.52 17 State income tax 17 State income tax 17 State income tax 17 State income tax Locality Correction Information 
Previously reported Correct information Previously reported Correct information 18 Local wages, tips, etc. 18 Local wages, tips, etc. 
18 Local wages, tips, etc. 18 Local wages, tips, etc. 19 Local income tax 19 Local income tax 19 Local income tax 19 Local income tax 20 
Locality name 20 Locality name 20 Locality name 20 Locality name Copy B—To Be Filed with Employee’s FEDERAL Tax Return Form W-2c (Rev. 8-2014) 
Corrected Wage and Tax Statement Department of the Treasury Internal Revenue Service"""

In [7]:
getNamedEntity(txt)

{'ORG': [{'index': [212, 228], 'value': 'FACET WEALTH INC'}],
 'PERSON': [{'index': [662, 675], 'value': 'DANIELLE COLE'}]}

### Azure Blob

In [1]:
from azure.storage.blob import BlockBlobService

In [2]:
ACCOUNT_NAME = "saacedeveastinput"
ACCOUNT_KEY = "RhJlnL3AloZbVZ48ENuRlYPBKCab1Wsoc94mbESIxtAmoI5ep3pxaTYhXgyAmDpvtPMpRbtJfV3A7YKKWSbKlQ==;EndpointSuffix=core.windows.net"
CONTAINER_NAME = "input"

In [7]:
referenceNo = 'ADR-2022-200000004738'
block_blob_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)
for file in block_blob_service.list_blobs(CONTAINER_NAME, referenceNo):
    print(file.name)

ADR-2022-200000004738/LoanMetaData.txt
ADR-2022-200000004738/Page-1.webp
ADR-2022-200000004738/pension_statement_27017.pdf
